# Angr Riscv

Interop between cbmc and angr?
frama-C?


In [ ]:
import angr
import claripy

# Load the binary
project = angr.Project('your_binary', auto_load_libs=False)

# Create an initial state with symbolic registers and memory
state = project.factory.blank_state()

# Symbolize registers if needed
eax = state.solver.BVS('eax', 32)
ebx = state.solver.BVS('ebx', 32)
state.regs.eax = eax
state.regs.ebx = ebx

# Symbolically execute the block of instructions
block = project.factory.block(state.addr)
irsb = block.vex
state = project.factory.entry_state(addr=state.addr)
simgr = project.factory.simgr(state)
simgr.step()

# Extract the final state and its constraints
final_state = simgr.active[0]
constraints = final_state.solver.constraints

# Print out the constraints (Z3 format)
for constraint in constraints:
    print(constraint)


# emulator

https://fmash16.github.io/content/posts/riscv-emulator-in-c.html
https://book.rvemu.app/index.html

https://github.com/cvut/qtrvsim
https://comparch.edu.cvut.cz/
https://www.iaik.tugraz.at/course/computer-organization-and-networks-inb06000uf-wintersemester-2023-24/
https://www.eit.lth.se/sprapport.php?uid=1361 customization of ibex
https://extgit.iaik.tugraz.at/con/examples-2021/-/blob/main/con06/riscv32-ibex-newlib-nano.cmake

eventually add bus for mmio



In [7]:
fibs = """
        .data
fibs:   .space 56

        .text
main:
        addi    s1, zero, 0     ; storage index
        addi    s2, zero, 56    ; last storage index
        addi    t0, zero, 1     ; t0 = F_{i}
        addi    t1, zero, 1     ; t1 = F_{i+1}
        ebreak                  ; launch debugger
loop:
        sw      t0, fibs(s1)    ; save
        add     t2, t1, t0      ; t2 = F_{i+2}
        addi    t0, t1, 0       ; t0 = t1
        addi    t1, t2, 0       ; t1 = t2
        addi    s1, s1, 4       ; increment storage pointer
        blt     s1, s2, loop    ; loop as long as we did not reach array length
        ebreak                  ; launch debugger
        ; exit gracefully
        addi    a0, zero, 0
        addi    a7, zero, 93
        scall                   ; exit with code 0
"""

hello_world = """
; hello-world.asm
; print "hello world" to stdout and exit
        .data
msg:    .ascii "Hello world\n"
        .text
        addi    a0, zero, 1             ; print to stdout
        addi    a1, zero, msg           ; load msg address
        addi    a2, zero, 12            ; write 12 bytes
        addi    a7, zero, SCALL_WRITE   ; write syscall code
        scall
        addi    a0, zero, 0             ; set exit code to 0
        addi    a7, zero, SCALL_EXIT    ; exit syscall code
        scall
"""

__
#from .core import CPU, ProgramLoader, Program, UserModeCPU
#from .instructions import InstructionSet, InstructionSetDict
#from .config import RunConfig
#from .helpers import FMT_GRAY, FMT_NONE
from riscemu.parser import AssemblyFileLoader
#from .instructions.float_base import FloatArithBase
#UserModeCpu
import io
f = io.StringIO(hello_world)
AssemblyFileLoader(f).load()

TypeError: ProgramLoader.__init__() missing 2 required positional arguments: 'source' and 'options'

https://github.com/AntonLydike/riscemu

run unicorn?


# Knuckle riscv

In [6]:
import knuckledragger as kd
from z3 import *

RVInsn = Datatype("RVInsn")
RVInsn.declare("ADD", ("rd", BitVecSort(5)), ("rs1", BitVecSort(5)), ("rs2", BitVecSort(5)))
RVInsn = RVInsn.create()

RVState = Datatype("RVState")
RVState.declare("mk", ("reg", ArraySort(BitVecSort(5), BitVecSort(32))), ("pc", BitVecSort(32)), ("mem", ArraySort(BitVecSort(32), BitVecSort(8))))
RVState = RVState.create()

insn = Const("insn", RVInsn)
st = Const("st", RVState)
addr = Const("addr", BitVecSort(32))

load32 = kd.define("load32", [st, addr], 
  Concat(Select(st.mem, addr + i) for i in range(4))
)
reg = Const("reg", )

regwrite = kd.define("regwrite", [st, reg, addr],
                       If(reg == 0, st, # r0 is not a real register.
                       
                       )
  Store(st.reg, insn.rd, load32(st, st.reg[insn.rs1]) + load32(st, st.reg[insn.rs2]))
)

exec_insn = kd.define("exec_insn", [insn,st],
  If(insn.is_ADD, 
                RVState.mk(
                Store(st.reg, insn.rd, st.reg[insn.rs1] + st.reg[insn.rs2]),
                st.pc + 1,
                st.mem
                ),
)

state0 = RVState.mk(K(0, BitVecSort(32)), K(0, BitVecSort(32)), K(0, BitVecSort(8)))

#st := If(insn.is_ADD, )






SyntaxError: '(' was never closed (488137330.py, line 21)

In [ ]:
RVInsn = Datatype("RVInsn")
RVInsn.declare("ADD", ("rd", BitVecSort(5)), ("rs1", BitVecSort(5)), ("rs2", BitVecSort(5)))
RVInsn.declare("SUB", ("rd", BitVecSort(5)), ("rs1", BitVecSort(5)), ("rs2", BitVecSort(5)))
RVInsn.declare("AND", ("rd", BitVecSort(5)), ("rs1", BitVecSort(5)), ("rs2", BitVecSort(5)))
RVInsn.declare("OR", ("rd", BitVecSort(5)), ("rs1", BitVecSort(5)), ("rs2", BitVecSort(5)))
RVInsn.declare("XOR", ("rd", BitVecSort(5)), ("rs1", BitVecSort(5)), ("rs2", BitVecSort(5)))
RVInsn.declare("SLL", ("rd", BitVecSort(5)), ("rs1", BitVecSort(5)), ("rs2", BitVecSort(5)))
RVInsn.declare("SRL", ("rd", BitVecSort(5)), ("rs1", BitVecSort(5)), ("rs2", BitVecSort(5)))
RVInsn.declare("SRA", ("rd", BitVecSort(5)), ("rs1", BitVecSort(5)), ("rs2", BitVecSort(5)))
RVInsn.declare("ADDI", ("rd", BitVecSort(5)), ("rs1", BitVecSort(5)), ("imm", BitVecSort(12)))
RVInsn.declare("LW", ("rd", BitVecSort(5)), ("rs1", BitVecSort(5)), ("imm", BitVecSort(12)))
RVInsn.declare("SW", ("rs1", BitVecSort(5)), ("rs2", BitVecSort(5)), ("imm", BitVecSort(12)))
RVInsn = RVInsn.create()

# Pcode knuckle

In [ ]:
# https://spinsel.dev/assets/2020-06-17-ghidra-brainfuck-processor-1/ghidra_docs/language_spec/html/pcoderef.html
# pcode2c
# angre code?

PcodeOp = EnumSort("PCodeOp", ["COPY", "LOAD", "STORE", "BRANCH", "INT_ADD" ...])

BinOp = EnumSort("PCodeBinOp", ["INT_EQUAL", ])
# I am very tempted to make size statically determined.
VarNode = kd.Record("VarNode", ("space", IntSort()), ("offset", IntSort()), ("size", IntSort()))


Insn = Datatype("Insn")
Insn.declare("binop", ("op", binop), ("out", Varnode), ("v1", VarNode), ("v2", VarNode))
Insn.declare("unop", ...)

Insns.create()

State = ArraySort(IntSort(), IntSort(), BitVecSort(8))

op = Const("op", PCodeOp)
exec_binop = kd.define(If(op == PCodeOp.COPY, 
   








In [ ]:
fetch = load32(state0, state0.pc)
state = 

In [ ]:
step = kd.define(
fetch = None;
decode = None;
pc += 4;
exec_insn(st)

In [8]:
print(x := 1 + 2; x + 3)

SyntaxError: invalid syntax (431848137.py, line 1)

In [ ]:
 1. a basic riscv python ionterpreter

Op = Enum("ADDI SLTI ANDI ORI XORI")

class Insn(op, a, b):



def biop(insn, a, b):
    match opcode:
        case Op.ADD:
            return a + b
        case Op.ANDI:
            return a & b
        case Op.ORI:
            return a | b

def step(pc, reg, mem, insn):
    res[insn.out] = biop(insn.op, reg[insn.a], reg[insn.b])
    pc += 1
    insn = mem[pc]


<https://riscv.org/wp-content/uploads/2017/05/riscv-spec-v2.2.pdf>

- Using define-fun-rec gives a first class handle on instructions.
- To curry or not.
-

```clojure
;re
; let's just stick to 32 bit operations
; 32 = 2^5 registers of 32 bits a piece
(define-sort REG () (_ BitVec 5))
(define-sort BV32 () (_ BitVec 32))
(define-sort REGFILE () (Array REG BV32))
(define-sort Addr () BV32)

(define-const r0 REG (_ bv0 5))
(define-const bv0 BV32 (_ bv0 32))
(define-const bv1 BV32 (_ bv1 32))


(declare-datatype State (
 (State
    (mem (Array Addr BV32))
    (regfile REGFILE)
    (pc BV32)
  ))
)

; r0 is always all zeros
(define-fun valid ((s State)) Bool
  (= (select (regfile s) r0) bv0)
)

; similarly for sub,and,or,xor
(define-fun add ((dst REG) (src1 REG) (src2 REG)) (Array REGFILE REGFILE Bool)
  (lambda ((in REGFILE) (out REGFILE))
      (= out (store in dst (bvadd (select in src1) (select in src2))))
  )
) 


(define-fun nop () (Array REGFILE REGFILE Bool)
  (lambda ((in REGFILE) (out REGFILE))
      (= out in)
  )
) 

```

```python
from z3 import *
from dataclasses import dataclass
REG = BitVecSort(5)
BV32 = BitVecSort(32)
MEM = ArraySort(BV32, BitVecSort(8))
REGFILE = ArraySort(REG, BV32) # could make this a python list. statically structured.


# Z3 records vs python records. An eternal battle.
# The state record is total static though. Shouldn't be a problem.
@dataclass
class RVState:
  mem: Array
  regfile: Array
  pc: BitVec

def FreshState():
  return RVState(FreshConst(MEM), FreshConst(REGFILE), FreshConst(BV32))

def add(dst,src1,src2):
  def res(in_,out):
    return out == Store(in_,dst, in_[src1] + in_[src2])
  return res

# functional style.
def add(dst,src1,src2):
  def res(s):
    return Store(s,dst, in_[src1] + in_[src2])
  return res

# relation composition.
# temporal compositin.
def comp(f,g):
  def res(in_,out):
    s = FreshState()
    return Exists([s.mem, s.regfile, s.pc], And( f(in_,s), g(s, out)))
  return res

def hcomp(f,g):
  def res(in_, out):
    return And(f(in_out), g(in_,out))
  return res

#foo = Array("foo", IntSort(),IntSort())
#print(foo["a"])
```

So what do I want to do with this model? WP? CHC? Model checking? Symbolic execution? Hoare Logic?
Bit accurate assembly modelling can be interesting too.

<https://github.com/mit-plv/riscv-coq/blob/master/src/riscv/Spec/Decode.v>

```z3
;re
(define-sort BV12 () (_ BitVec 12))
(define-sort BV32 () (_ BitVec 32))
(define-sort Reg  () (_ BitVec 5)) ; 32 = 2 ^ 5 registers
(declare-datatype IOp (
  (ADDI) (SLLI) (SLTI) (SLTIU) (XORI) (SRLI) (SRAI) (ORI) (ANDI)
))

(declare-datatype ROp (
  (ADD) (SLL) (SLT) (SLTU) (XOR) (SRL) (SRA) (OR) (AND)
))

(declare-datatype Instruction (
 (IInsn (opI IOp) (rdI Reg) (rs1I Reg) (immI BV12))
 (RInsn (opR ROp) (rdR Reg) (rs1R Reg) (rs2R Reg))
 ))

;(simplify (IInsn AddI #b00000 #b00000 #b000000000000))

(define-const opcode_OP_IMM (_ BitVec 7) #b0010011)
(define-const opcode_OP  (_ BitVec 7) #b0110011)
(define-fun funct3 ((op IOp)) (_ BitVec 3)
 (match op (
      (ADDI   #b000)
      (SLLI   #b001)
      (SLTI   #b010)
      (SLTIU  #b011)
      (XORI   #b100)
      (SRLI   #b101)
      (SRAI   #b101)
      (ORI    #b110)
      (ANDI   #b111)
   ))
)


(define-fun funct3 ((op ROp)) (_ BitVec 3)
 (match op (
      (ADD   #b000)
      (SUB   #b000)
      (SLL   #b001)
      (SLT   #b010)
      (SLTU  #b011)
      (XOR   #b100)
      (SRL   #b101)
      (SRA   #b101)
      (OR    #b110)
      (AND   #b111)
   ))
)

(define-fun funct7 ((op ROp)) (_ BitVec 7)
 (match op (
      (ADD   #b0000000)
      (SUB   #b0100000)
      (SLL   #b0000000)
      (SLT   #b0000000)
      (SLTU  #b0000000)
      (XOR   #b0000000)
      (SRL   #b0000000)
      (SRA   #b0100000)
      (OR    #b0000000)
      (AND   #b0000000)
   ))
)

; encode is also decode since SMT is declarative. That's nices.
(define-fun encode ((insn Instruction)) BV32
  (match insn (
    ((IInsn op dst src imm) (concat imm src (funct3 op) dst opcode_OP_IMM))
    ((RInsn op rd rs1 rs2)  (concat (funct7 op) rs2 rs1 (funct3 op) rd opcode_OP))
  ))
)

(define-sort REGFILE () (Array Reg BV32))
(declare-datatype State (
 (State
    ;(mem (Array Addr BV32))
    (regfile REGFILE)
    (pc BV32)
  ))
)


(define-fun execI ((op IOp) (rs1 BV32) (imm BV12)) BV32
(let (
       (imm ((_ zero_extend 20) imm))
     )
 (match op (
  ; todo fill in correctly
      (ADDI   (bvadd rs1 imm))
      (SLLI   imm)
      (SLTI   imm)
      (SLTIU  imm)
      (XORI   (bvxor rs1 imm))
      (SRLI   imm)
      (SRAI   imm)
      (ORI    (bvor rs1 imm))
      (ANDI   (bvand rs1 imm))
   ))
))

(define-fun execR ((op ROp) (rs1 BV32) (rs2 BV32)) BV32
 (match op (
  ; todo fill in correctly
      (ADD   (bvadd rs1 rs2))
      (SUB   (bvsub rs1 rs2))
      (SLL   rs1)
      (SLT   rs1)
      (SLTU  rs1)
      (XOR   (bvxor rs1 rs2))
      (SRL   rs1)
      (SRA   rs1)
      (OR    (bvor rs1 rs2))
      (AND   (bvand rs1 rs2))
   ))
)

(define-fun inc-pc ((st State)) State
  ((_ update-field pc) st (bvadd (pc st) (_ bv1 32)))
)

(define-fun set-reg ((st State) (reg Reg) (v BV32)) State
  ((_ update-field regfile) st (store (regfile st) reg v))
)

(define-fun get-reg ((st State) (reg Reg)) BV32
  (select (regfile st) reg)
)

(define-fun execute ((insn Instruction) (st State)) State
    (match insn (
    ((IInsn op dst src imm)  (inc-pc (set-reg st dst (execI op (get-reg st src) imm))))
    ((RInsn op rd rs1 rs2)   (inc-pc (set-reg st rd (execR op (get-reg st rs1) (get-reg st rs2)))))
  ))
)

(define-const zero32 BV32 (_ bv0 32))

(define-const init-regfile (Array Reg BV32)
((as const (Array Reg BV32)) zero32))

(define-const init-state State 
  (State init-regfile (_ bv0 32))
)

(define-const r0 Reg #b00000)
(define-const r1 Reg #b00001)
(define-const r2 Reg #b00010)
(simplify (execute (IInsn ADDI r2 r1 (_ bv4 12)) init-state))
```

```z3
; scratch pad
(simplify (concat #xf0 #x00))
```